# Fastest Lap Data
A Jupyter notebook looking at the fastest laps from each race. Using data from [Formula 1](https://www.formula1.com/en/results.html/2022/races.html)

The DataFrame 'fastest_lap' contains the mean (average) of the fastest laps of each driver in each race.

CSV files can be found in:
- data/fastest-laps

__HAVE YOU READ THE README FILE? PLEASE DO BEFORE USING THIS JUPYTER NOTEBOOK!__

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import cufflinks as cf
import re
from datetime import datetime
import time

In [2]:
%matplotlib inline

In [3]:
cf.go_offline() #allows to use cufflinks offline

In [33]:
fastest_lap = pd.DataFrame()

fastest_lap = fastest_lap.assign(NO = '', DRIVER = '', RACE = '', FL = '', AVG_SPEED = '')
# rename the column header
#fastest_lap.columns = fastest_lap.columns.str.replace('SPEED', 'AVG SPEED')

fastest_lap.head()

,NO,DRIVER,RACE,FL,AVG_SPEED


In [34]:
fl_bahrain = pd.read_csv('data/fastest-lap/fl-bahrain.csv')
fl_bahrain.head()

,POS,NO,DRIVER,CAR,LAP,TIME OF DAY,TIME,AVG SPEED
0,1,16,Charles Leclerc,FERRARI,51,19:31:35,01:34.6,206.018
1,2,1,Max Verstappen,RED BULL RACING RBPT,51,19:31:37,01:35.4,204.140
2,3,55,Carlos Sainz,FERRARI,52,19:33:13,01:35.7,203.501
3,4,11,Sergio Perez,RED BULL RACING RBPT,52,19:33:14,01:36.1,202.762
4,5,44,Lewis Hamilton,MERCEDES,53,19:34:51,01:36.2,202.469


In [35]:
fl_bahrain.describe()

,POS,NO,LAP,AVG SPEED
count,20.00000,20.000000,20.000000,20.000000
mean,10.50000,25.800000,49.050000,201.422450
std,5.91608,21.142623,5.889107,1.812899
min,1.00000,1.000000,34.000000,198.300000
25%,5.75000,10.750000,49.000000,200.495750
50%,10.50000,21.000000,51.000000,201.180000
75%,15.25000,34.250000,53.000000,202.352000
max,20.00000,77.000000,56.000000,206.018000


In [36]:
# get the fastest lap of the race
fastest_lap_bahrain = fl_bahrain.loc[0, 'TIME']
fastest_lap_bahrain_time = datetime.strptime(fastest_lap_bahrain,'%M:%S.%f')
fastest_lap_bahrain_no = fl_bahrain.loc[0, 'NO']
fastest_lap_bahrain_driver = fl_bahrain.loc[0, 'DRIVER']
fastest_lap_bahrain_speed = fl_bahrain.loc[0, 'AVG SPEED']

fastest_lap_bahrain_time.strftime('%M:%S.%f')[:-3]

'01:34.600'

In [37]:
# add fastest driver
# add row
fastest_lap.loc[-1] = [fastest_lap_bahrain_no, fastest_lap_bahrain_driver, 'BAHRAIN', fastest_lap_bahrain_time.strftime('%M:%S.%f')[:-3], fastest_lap_bahrain_speed]
# shift the index
fastest_lap.index = fastest_lap.index + 1
fastest_lap.head()

,NO,DRIVER,RACE,FL,AVG_SPEED
0,16,Charles Leclerc,BAHRAIN,01:34.600,206.018


In [38]:
fl_bahrain_all_drivers = []

driver_count = 1

while driver_count < 20:
    time = fl_bahrain.loc[driver_count, 'TIME']
    driverNo = fl_bahrain.loc[driver_count, 'NO']
    name = fl_bahrain.loc[driver_count, 'DRIVER']
    avg_speed = fl_bahrain.loc[driver_count, 'AVG SPEED']
    if time == 'DNF':
        driver_count += 1
    else:
        #time = re.sub(r'[^0-9.]', '', time)
        #timeFloat = float(time)
        formattedTime = datetime.strptime(time,'%M:%S.%f')
        #finalTime = timeFloat + fastest_lap_bahrain_time.strftime('%M:%S.%f')[:-3]
        # add row
        fastest_lap.loc[-1] = [driverNo, name, 'BAHRAIN', formattedTime.strftime('%M:%S.%f')[:-3], avg_speed]
        # shift the index
        fastest_lap.index = fastest_lap.index + 1
        driver_count += 1

fastest_lap.head()

,NO,DRIVER,RACE,FL,AVG_SPEED
19,16,Charles Leclerc,BAHRAIN,01:34.600,206.018
18,1,Max Verstappen,BAHRAIN,01:35.400,204.140
17,55,Carlos Sainz,BAHRAIN,01:35.700,203.501
16,11,Sergio Perez,BAHRAIN,01:36.100,202.762
15,44,Lewis Hamilton,BAHRAIN,01:36.200,202.469


In [43]:
fastest_lap.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 19 to 0
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   NO         20 non-null     int64  
 1   DRIVER     20 non-null     object 
 2   RACE       20 non-null     object 
 3   FL         20 non-null     object 
 4   AVG_SPEED  20 non-null     float64
dtypes: float64(1), int64(1), object(3)
memory usage: 960.0+ bytes


In [46]:
# convert column to datetime
fastest_lap['FL'] = pd.to_datetime(fastest_lap['FL'])

In [48]:
fastest_lap.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 19 to 0
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   NO         20 non-null     int64         
 1   DRIVER     20 non-null     object        
 2   RACE       20 non-null     object        
 3   FL         20 non-null     datetime64[ns]
 4   AVG_SPEED  20 non-null     float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 960.0+ bytes


In [49]:
fastest_lap.describe()

,NO,AVG_SPEED
count,20.000000,20.000000
mean,25.800000,201.422450
std,21.142623,1.812899
min,1.000000,198.300000
25%,10.750000,200.495750
50%,21.000000,201.180000
75%,34.250000,202.352000
max,77.000000,206.018000


In [ ]:
fl_saudi_arabia = pd.read_csv('data/fastest-lap/fl-saudi-arabia.csv')
fl_saudi_arabia.head()

,POS,NO,DRIVER,CAR,LAP,TIME OF DAY,TIME,AVG SPEED
0,1,16,Charles Leclerc,FERRARI,48,21:24:43,01:31.6,242.556
1,2,1,Max Verstappen,RED BULL RACING RBPT,50,21:27:46,01:31.8,242.191
2,3,55,Carlos Sainz,FERRARI,48,21:24:50,01:31.9,241.841
3,4,11,Sergio Perez,RED BULL RACING RBPT,46,21:21:47,01:32.0,241.481
4,5,4,Lando Norris,MCLAREN MERCEDES,46,21:22:26,01:32.8,239.629


In [ ]:
fl_australia = pd.read_csv('data/fastest-lap/fl-australia.csv')
fl_australia.head()

,POS,NO,DRIVER,CAR,LAP,TIME OF DAY,TIME,AVG SPEED
0,1,16,Charles Leclerc,FERRARI,58,16:30:59,01:20.3,236.740
1,2,14,Fernando Alonso,ALPINE RENAULT,57,16:31:30,01:20.8,235.024
2,3,11,Sergio Perez,RED BULL RACING RBPT,58,16:31:20,01:21.1,234.305
3,4,63,George Russell,MERCEDES,53,16:24:35,01:21.5,233.152
4,5,77,Valtteri Bottas,ALFA ROMEO FERRARI,54,16:26:39,01:21.7,232.707
